# Modeling

In [1]:
import datetime

import tensorflow as tf
from tensorflow.keras import layers

from src.dataset import SegmentationDataset
from src.data_pipeline import SegmentationDataPipeline


%load_ext lab_black
%load_ext autoreload
%autoreload 2

## Load data and create pipeline

In [2]:
IMG_SHAPE = (256, 1600)
BATCH_SIZE = 8
ANNOTATIONS_PATH = "../data/train.csv"
TRAIN_IMG_PATH = "../data/train_images/"

# instantiate dataset and pipelne
sd = SegmentationDataset(label_file=ANNOTATIONS_PATH, img_dir_path=TRAIN_IMG_PATH)

# create train/test & x/y splits
train_imgs, test_imgs = sd.get_train_test_split(test_size=0.2)
X_train, y_train = sd.get_image_sequence(train_imgs[:50]), sd.get_label_sequence(
    train_imgs[:50]
)
X_test, y_test = sd.get_image_sequence(test_imgs[:50]), sd.get_label_sequence(
    test_imgs[:50]
)

# create dataset pipelines
sdp = SegmentationDataPipeline(
    img_shape=IMG_SHAPE,
    pipeline_options={
        "map_parallel": None,
        "cache": False,
        "shuffle_buffer_size": False,
        "batch_size": 4,
        "prefetch": False,
    },
)
train_dataset = sdp(X_train, y_train)
test_dataset = sdp(X_test, y_test)

TypeError: __init__() missing 1 required positional argument: 'img_shape'

## Define Model 

In [3]:
from keras.models import Model
from keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    UpSampling2D,
    Concatenate,
    Conv2DTranspose,
    BatchNormalization,
    Dropout,
    Lambda,
    Activation,
)

In [4]:
def conv_block(x, n_filters):
    x = Conv2D(n_filters, kernel_size=3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(n_filters, kernel_size=3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x


def encoder_block(x, n_filters):
    x = conv_block(x, n_filters)
    p = MaxPooling2D(pool_size=(2, 2))(x)
    p = Dropout(rate=0.3)(p)
    return x, p


def decoder_block(x, skip_features, n_filters):
    x = Conv2DTranspose(n_filters, kernel_size=(2, 2), strides=2, padding="same")(x)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, n_filters)
    return x


def unet(input_shape):

    input_shape = input_shape + (3,)
    inputs = Input(input_shape)

    # encoder downsampling (x4)
    c1, p1 = encoder_block(inputs, 64)
    c2, p2 = encoder_block(p1, 128)
    c3, p3 = encoder_block(p2, 256)
    c4, p4 = encoder_block(p3, 512)

    # bottleneck
    b = conv_block(p4, 1024)

    # decoder upsampling (x4)
    d1 = decoder_block(b, c4, 512)
    d2 = decoder_block(d1, c3, 256)
    d3 = decoder_block(d2, c2, 128)
    d4 = decoder_block(d3, c1, 64)

    outputs = Conv2D(filters=4, kernel_size=1, padding="same", activation="softmax")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model

In [5]:
unet = unet(IMG_SHAPE)

In [6]:
# unet.summary()

## Train Model

In [7]:
unet.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy"],
)

log_dir = "../logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("test_segmentation.h5", save_best_only=True),
    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1),
]

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [8]:
EPOCHS = 5
unet.fit(
    train_dataset, epochs=EPOCHS, validation_data=test_dataset, callbacks=callbacks
)

Epoch 1/5


2022-09-28 14:08:38.271398: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.0KiB (rounded to 1024)requested by op Fill
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-09-28 14:08:38.271592: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-09-28 14:08:38.271644: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 88, Chunks in use: 88. 22.0KiB allocated for chunks. 22.0KiB in use in bin. 10.2KiB client-requested in use in bin.
2022-09-28 14:08:38.271679: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 40, Chunks in use: 40. 20.0KiB allocated for chunks. 20.0KiB in use in bin. 20.0KiB client-requested in use in bin.
2022-09-28 14:08:38.271712: I tensorflo

ResourceExhaustedError: in user code:

    File "/home/cdsw/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/cdsw/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/cdsw/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/cdsw/.local/lib/python3.9/site-packages/keras/engine/training.py", line 863, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/home/cdsw/.local/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py", line 532, in minimize
        return self.apply_gradients(grads_and_vars, name=name)
    File "/home/cdsw/.local/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py", line 639, 